In [1]:
!pip3 install openpyxl
!pip3 install matplotlib
!pip3 install h2o
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import h2o
from h2o.automl import H2OAutoML
h2o.init()

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-co

H2O_cluster_uptime:,05 secs
H2O_cluster_timezone:,Asia/Jakarta
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.0.3
H2O_cluster_version_age:,3 months and 10 days
H2O_cluster_name:,H2O_from_python_sreeya_02dr3d
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [2]:
data = pd.read_excel("MI_dataset_final.xlsx")

In [3]:
data_f = h2o.H2OFrame(data)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [4]:
train, valid = data_f.split_frame(ratios = [.9])

In [5]:
testing_data = pd.read_excel("/Users/sreeya/Documents/Uni/Thesis/cleaned/cleaned.xlsx")

In [6]:
testing_data = testing_data.dropna()

In [7]:
testing_data.isnull().sum()

Unnamed: 0             0
Wellbore ID            0
Date                   0
Hours Online           0
Av. WHT (Deg C)        0
Av. WHP (bar)          0
Av. DHT (Deg C)        0
Av. DHP (bar)          0
Oil (m3)               0
Gas (m3)               0
Produced Water (m3)    0
dtype: int64

In [8]:
data_pred = h2o.H2OFrame(testing_data)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [9]:
x = train.columns
y = "Oil (m3)"
x.remove(y)
x.remove('Gas (m3)')

In [10]:
aml = H2OAutoML(max_runtime_secs = 1800, sort_metric = "RMSE",nfolds = 0)
aml.train(x=x, y=y, training_frame=train,validation_frame=valid)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OXGBoostEstimator :  XGBoost
Model Key:  XGBoost_lr_search_selection_AutoML_1_20220527_174933_select_grid_model_2


Model Summary: 


,,number_of_trees
0,,41.0




ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 185.73076177393932
RMSE: 13.628307370100636
MAE: 5.86719530699233
RMSLE: 0.07981547992488562
Mean Residual Deviance: 185.73076177393932

ModelMetricsRegression: xgboost
** Reported on validation data. **

MSE: 14120.40115700415
RMSE: 118.82929418709912
MAE: 40.01004688474047
RMSLE: 0.2826076927929805
Mean Residual Deviance: 14120.40115700415

Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
0,,2022-05-27 18:19:10,2.127 sec,0.0,1078.125258,689.606176,1.162354e+06,1089.271266,698.850039,1.186512e+06
1,,2022-05-27 18:19:11,2.482 sec,10.0,144.248718,82.716237,2.080769e+04,183.912495,98.755123,3.382381e+04
2,,2022-05-27 18:19:11,2.937 sec,20.0,34.398838,16.732193,1.183280e+03,122.573220,44.822381,1.502419e+04
3,,2022-05-27 18:19:12,3.487 sec,30.0,17.621573,7.932392,3.105198e+02,119.283724,40.455577,1.422861e+04
4,,2022-05-27 18:19:12,3.998 sec,40.0,13.809420,5.976940,1.907001e+02,118.852537,40.012916,1.412593e+04
5,,2022-05-27 18:19:12,4.136 sec,41.0,13.628307,5.867195,1.857308e+02,118.829294,40.010047,1.412040e+04



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Av. DHT (Deg C),1.073877e+10,1.000000,0.335378
1,Av. DHP (bar),8.451279e+09,0.786988,0.263938
2,Av. WHT (Deg C),6.405380e+09,0.596472,0.200044
3,Av. WHP (bar),5.746421e+09,0.535110,0.179464
4,Hours Online,6.780637e+08,0.063142,0.021176


In [11]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,rmse,mean_residual_deviance,mse,mae,rmsle
XGBoost_lr_search_selection_AutoML_1_20220527_174933_select_grid_model_2,118.829,14120.4,14120.4,40.01,0.282608
GBM_grid_1_AutoML_1_20220527_174933_model_41,119.796,14351.2,14351.2,38.5503,0.278144
GBM_grid_1_AutoML_1_20220527_174933_model_132,124.1,15400.7,15400.7,47.8801,nan
XGBoost_grid_1_AutoML_1_20220527_174933_model_14,124.508,15502.2,15502.2,41.5553,0.275699
GBM_grid_1_AutoML_1_20220527_174933_model_46,124.565,15516.3,15516.3,45.7725,0.276049
GBM_grid_1_AutoML_1_20220527_174933_model_88,125.354,15713.7,15713.7,41.9154,nan
GBM_grid_1_AutoML_1_20220527_174933_model_58,125.846,15837.3,15837.3,40.1746,nan
XGBoost_grid_1_AutoML_1_20220527_174933_model_24,127.694,16305.9,16305.9,45.4494,nan
GBM_grid_1_AutoML_1_20220527_174933_model_63,128.191,16433,16433,40.8609,nan
GBM_grid_1_AutoML_1_20220527_174933_model_99,128.773,16582.6,16582.6,41.8343,nan


In [12]:
h = h2o.get_model("GBM_grid_1_AutoML_1_20220527_174933_model_41")

In [13]:
preds = h.predict(data_pred)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [14]:
preds

predict
618.742
703.262
797.682
519.634
761.582
745.342
734.594
664.534
513.34
478.542


In [15]:
data_as_list = h2o.as_list(preds, use_pandas=False)
pr = h2o.as_list(preds)

In [16]:
from sklearn.metrics import mean_squared_error
from math import sqrt

#calculate RMSE
sqrt(mean_squared_error(pr['predict'], testing_data['Oil (m3)'])) 

551.4845800311904

In [17]:
h2 = h2o.get_model("DRF_1_AutoML_1_20220527_174933")

In [18]:
preds2 = h2.predict(data_pred)

drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [19]:
preds2

predict
698.309
914.955
950.846
882.058
964.436
705.59
931.569
726.021
743.813
713.862


In [20]:
data_as_list2 = h2o.as_list(preds2, use_pandas=False)
pr2 = h2o.as_list(preds2)

In [21]:
from sklearn.metrics import mean_squared_error
from math import sqrt

#calculate RMSE
sqrt(mean_squared_error(pr2['predict'], testing_data['Oil (m3)'])) 

401.53338336923156

# Gas

In [22]:
x1 = train.columns
y1 = "Gas (m3)"
x1.remove(y1)
x1.remove('Oil (m3)')

In [23]:
aml = H2OAutoML(max_runtime_secs = 1800, sort_metric = "RMSE",nfolds = 0)
aml.train(x=x1, y=y1, training_frame=train,validation_frame=valid)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_grid_1_AutoML_2_20220527_183626_model_151


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,75.0,75.0,583660.0,15.0,15.0,15.0,98.0,1033.0,615.9067




ModelMetricsRegression: gbm
** Reported on train data. **

MSE: 232575991.0302195
RMSE: 15250.44232244493
MAE: 5884.324722282547
RMSLE: NaN
Mean Residual Deviance: 232575991.0302195

ModelMetricsRegression: gbm
** Reported on validation data. **

MSE: 1779759225.112857
RMSE: 42187.19266688478
MAE: 13967.388497037367
RMSLE: NaN
Mean Residual Deviance: 1779759225.112857

Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_mae,training_deviance,validation_rmse,validation_mae,validation_deviance
0,,2022-05-27 18:57:33,6 min 53.665 sec,0.0,154860.383570,112097.038164,2.398174e+10,158827.696234,113870.722484,2.522624e+10
1,,2022-05-27 18:57:33,6 min 53.879 sec,5.0,96987.368651,68534.556611,9.406550e+09,102751.468480,70591.588311,1.055786e+10
2,,2022-05-27 18:57:33,6 min 54.069 sec,10.0,63938.785937,42989.563727,4.088168e+09,72066.710682,45498.191395,5.193611e+09
3,,2022-05-27 18:57:33,6 min 54.312 sec,15.0,44878.352261,27805.798844,2.014067e+09,56947.122569,31400.979989,3.242975e+09
4,,2022-05-27 18:57:33,6 min 54.525 sec,20.0,34206.176378,18904.355505,1.170063e+09,49008.855117,23294.450885,2.401868e+09
5,,2022-05-27 18:57:34,6 min 54.666 sec,25.0,27965.532011,13686.046979,7.820710e+08,45342.712926,18892.479485,2.055962e+09
6,,2022-05-27 18:57:34,6 min 54.868 sec,30.0,24277.357081,10663.557657,5.893901e+08,43702.157467,16485.309168,1.909879e+09
7,,2022-05-27 18:57:34,6 min 55.075 sec,35.0,22030.995432,8943.709671,4.853648e+08,43418.973442,15352.541265,1.885207e+09
8,,2022-05-27 18:57:34,6 min 55.202 sec,40.0,20327.078570,7938.387014,4.131901e+08,42932.912304,14718.474591,1.843235e+09
9,,2022-05-27 18:57:34,6 min 55.384 sec,45.0,19135.731371,7318.928736,3.661762e+08,42638.952963,14396.408488,1.818080e+09



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,Av. WHP (bar),7.963188e+14,1.000000,0.422857
1,Av. WHT (Deg C),4.121769e+14,0.517603,0.218872
2,Av. DHT (Deg C),3.464027e+14,0.435005,0.183945
3,Av. DHP (bar),2.558351e+14,0.321272,0.135852
4,Hours Online,7.245158e+13,0.090983,0.038473


In [24]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,rmse,mean_residual_deviance,mse,mae,rmsle
GBM_grid_1_AutoML_2_20220527_183626_model_151,42187.2,1.77976e+09,1.77976e+09,13967.4,nan
GBM_grid_1_AutoML_2_20220527_183626_model_115,42732.8,1.82609e+09,1.82609e+09,13410.7,0.391121
XGBoost_grid_1_AutoML_2_20220527_183626_model_75,42979.5,1.84723e+09,1.84723e+09,15496.8,nan
GBM_grid_1_AutoML_2_20220527_183626_model_33,43157.5,1.86257e+09,1.86257e+09,15001.3,nan
GBM_grid_1_AutoML_2_20220527_183626_model_30,43225.3,1.86843e+09,1.86843e+09,14831,nan
GBM_grid_1_AutoML_2_20220527_183626_model_55,43649.8,1.9053e+09,1.9053e+09,15283.2,nan
XGBoost_grid_1_AutoML_2_20220527_183626_model_6,43924.5,1.92936e+09,1.92936e+09,16323.5,nan
GBM_grid_1_AutoML_2_20220527_183626_model_43,44209.5,1.95448e+09,1.95448e+09,14368.5,0.411841
XGBoost_grid_1_AutoML_2_20220527_183626_model_98,44600.1,1.98917e+09,1.98917e+09,16834.5,nan
GBM_grid_1_AutoML_2_20220527_183626_model_148,44788.1,2.00597e+09,2.00597e+09,17242,nan


In [25]:
h = h2o.get_model("GBM_grid_1_AutoML_2_20220527_183626_model_151")

In [26]:
preds = h.predict(data_pred)

gbm prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [27]:
data_as_list = h2o.as_list(preds, use_pandas=False)
pr = h2o.as_list(preds)

In [28]:
pr

,predict
0,166862.503311
1,73698.025650
2,117096.497596
3,43469.095759
4,77303.049909
...,...
505,129422.506705
506,54096.382550
507,53493.700032
508,54952.027615


In [29]:
from sklearn.metrics import mean_squared_error
from math import sqrt

#calculate RMSE
sqrt(mean_squared_error(pr['predict'], testing_data['Gas (m3)'])) 

156021.98908827736

In [30]:
h2 = h2o.get_model("DRF_1_AutoML_2_20220527_183626")

In [31]:
preds2 = h2.predict(data_pred)

drf prediction progress: |███████████████████████████████████████████████████████| (done) 100%


In [32]:
preds2

predict
220248
221351
233153
252653
225800
273856
236698
290996
255675
259666


In [33]:
data_as_list2 = h2o.as_list(preds2, use_pandas=False)
pr2 = h2o.as_list(preds2)

In [34]:
from sklearn.metrics import mean_squared_error
from math import sqrt

#calculate RMSE
sqrt(mean_squared_error(pr2['predict'], testing_data['Gas (m3)'])) 

146485.41914077871